# Sharpe optimal portfolio - comparisons

This example compares the performance of several risk-Sharpe optimal portfolios. 

The CVaR, SMCR, EVaR, MAD, LSD, BTAD, BTSD, GINI, SD and MV risk-based portfolios are considered. We use the `rtype=Sharpe` optimization flag with a risk-free rate of 0 (default value).

We start by importing **azapy** and other useful packages.

In [1]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.2.3


### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `force=True` get historical market data directly from the provider (in this case yahoo)
- `save=False` the historical market data is not saved

>For more information see https://azapy.readthedocs.io/en/latest/.

In [2]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'VHT']

sdate = "2012-01-01"
edate = "today"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, force=True, save=False)

get GLD from yahoo only
get TLT from yahoo only
get XLV from yahoo only
get VGT from yahoo only
get VHT from yahoo only

Request between 2012-01-03 : 2024-03-28
                    GLD         TLT         XLV         VGT         VHT
source            yahoo       yahoo       yahoo       yahoo       yahoo
force              True        True        True        True        True
save              False       False       False       False       False
file_dir         outDir      outDir      outDir      outDir      outDir
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
nrow               3079        3079        3079        3079        3079
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2024-03-28  2024-03-28  2024-03-28  2024-03-28  2024-03-28
error                No          No          No          No          No
extraction time 2.057 s


### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio,
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR` equal weighted,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR` equal weighted,
- `'P_EVaR'` is the mEVaR optimal portfolio with confidence levels `alpha_EVaR` equal weighted.
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSD'` is the mLSD optimal portfolio with weights `coef_LSD`,
- `'P_Omega'` is the mBTAD optimal portfolio with threshold levels `alpha_BTAD` equal weighted,
- `'P_Sortino'` is the mBTSD optimal portfolio with threshold levels `alpha_BTSD` equal weighted,
- `'P_GINI'` is the Gini optimal portfolio (commented out in `models` dictionary below, for speed reasons),
- `'P_MV'` is the MV optimal portfolio,
- `'P_SD'` is the SD optimal portfolio.


- In all cases the optimization type is set to `rtype='Sharpe'` (optimal portfolio with the same risk profile as the equal weighted portfolio) with risk-free rate mu0=0.
- The `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` where it is set to 1.25 years (for computational speed convenience).

> Note:
> - mBTAD-Sharpe ratio is called Omega ratio 
> - mBTSD-Sharpe ratio is called Sortino ratio 
> - MV-Sharpe ratio is different than (SD) Sharp ratio 

In [3]:
rtype = 'Sharpe'
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
coef_MAD = [1./3.] * 3
coef_LSD = [1./3.] * 3
alpha_BTAD = [-0.01, 0]
alpha_BTSD = [-0.01, 0]
alpha_EVaR = [0.75, 0.65]
hlength = 1.25

models = {'P_CVaR-Sharpe': {'type': 'Port_CVaR', 'm_param': {'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength}},
          'P_SMCR-Sharpe': {'type': 'Port_SMCR', 'm_param': {'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength}},
          'P_EVaR-Sharpe': {'type': 'Port_EVaR', 'm_param': {'alpha': alpha_EVaR, 'rtype': rtype, 'hlength': hlength}},
          'P_MAD-Sharpe': {'type': 'Port_MAD', 'm_param': {'coef': coef_MAD, 'rtype': rtype, 'hlength': hlength}},
          'P_LSD-Sharpe': {'type': 'Port_LSD', 'm_param': {'coef': coef_LSD, 'rtype': rtype, 'hlength': hlength}},
          'P_Omega': {'type': 'Port_BTAD', 'm_param': {'alpha': alpha_BTAD, 'rtype': rtype, 'hlength': hlength}},
          'P_Sortino': {'type': 'Port_BTSD', 'm_param': {'alpha': alpha_BTSD, 'rtype': rtype, 'hlength': hlength}},
          #'P_GINI-Sharpe': {'type': 'Port_GINI', 'm_param': {'rtype': rtype, 'hlength': 1.25}},
          'P_MV-Sharpe': {'type': 'Port_MV', 'm_param': {'rtype': rtype, 'hlength': hlength}},
          'P_Sharpe': {'type': 'Port_SD', 'm_param': {'rtype': rtype, 'hlength': hlength}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [4]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [5]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [6]:
_ = ps.port_view_all(componly=True, fancy=True, title="Relative performance")

### Portfolio performances

In [7]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
P_MV-Sharpe,9.26,-17.19,0.538737,2022-09-30,2021-12-30,2024-03-01,792
P_N,10.07,-21.29,0.472815,2022-10-20,2021-12-30,2024-02-07,769
P_Sortino,9.33,-20.87,0.447253,2018-12-24,2018-10-03,2019-08-08,309
P_CVaR-Sharpe,9.87,-23.95,0.412110,2018-12-24,2018-10-03,2019-08-27,328
P_LSD-Sharpe,9.83,-23.95,0.410444,2018-12-24,2018-10-03,2019-09-04,336
P_EVaR-Sharpe,9.66,-23.95,0.403199,2018-12-24,2018-10-03,2019-12-27,450
P_Sharpe,9.50,-23.95,0.396761,2018-12-24,2018-10-03,2019-08-14,315
P_MAD-Sharpe,9.08,-23.63,0.384158,2018-12-24,2018-10-03,2019-08-15,316
P_SMCR-Sharpe,8.93,-23.95,0.372725,2018-12-24,2018-10-03,2020-01-02,456


### Annual returns

Note the best and worst performer each year.

> The flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.

In [8]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_CVaR-Sharpe,P_EVaR-Sharpe,P_LSD-Sharpe,P_MAD-Sharpe,P_MV-Sharpe,P_N,P_Omega,P_SMCR-Sharpe,P_Sharpe,P_Sortino
year,,,,,,,,,,
2015,-8.13%,-7.67%,-8.11%,-7.47%,-6.57%,-3.17%,-7.06%,-7.40%,-7.57%,-8.01%
2016,2.95%,4.01%,4.18%,2.16%,0.18%,4.65%,2.08%,2.80%,2.02%,3.73%
2017,30.77%,31.13%,31.19%,34.39%,31.26%,20.34%,34.31%,30.43%,33.56%,32.32%
2018,3.65%,4.47%,3.62%,1.15%,5.19%,5.80%,-2.92%,5.10%,2.54%,2.29%
2019,18.02%,15.74%,16.86%,17.55%,17.65%,22.23%,17.77%,14.73%,18.38%,17.83%
2020,30.96%,28.26%,29.80%,26.95%,22.92%,26.33%,25.65%,21.80%,28.26%,27.89%
2021,13.31%,13.91%,14.00%,12.28%,12.25%,12.72%,12.05%,14.91%,11.64%,13.12%
2022,-5.62%,-6.78%,-6.32%,-7.63%,-6.51%,-14.79%,-8.49%,-6.79%,-6.97%,-7.34%
2023,0.72%,1.29%,1.17%,0.95%,2.40%,15.03%,1.38%,0.91%,1.76%,0.33%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [9]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_Omega portfolio

In [10]:
pp['P_Omega'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
month,,,,,,,,,,
1,nan%,-3.94%,4.72%,7.19%,5.10%,4.28%,-1.92%,-7.47%,-1.83%,0.36%
2,nan%,-0.04%,4.28%,-1.54%,2.01%,1.02%,-2.93%,-3.06%,-4.64%,2.74%
3,nan%,5.04%,1.64%,-0.26%,-1.98%,4.34%,-1.52%,5.51%,1.65%,3.88%
4,nan%,-0.94%,2.34%,-0.12%,-1.57%,6.44%,3.60%,-4.47%,2.24%,nan%
5,nan%,0.60%,4.41%,3.11%,1.06%,1.58%,0.12%,-0.57%,-3.12%,nan%
6,-1.90%,9.97%,-5.06%,-2.01%,5.66%,-0.62%,3.61%,-2.26%,3.77%,nan%
7,2.71%,3.25%,4.13%,0.76%,-0.58%,4.96%,4.80%,0.98%,2.35%,nan%
8,-7.87%,-0.33%,3.11%,5.12%,5.96%,0.74%,2.41%,-4.71%,-1.74%,nan%
9,-9.67%,-0.57%,2.75%,-0.27%,-3.92%,-1.20%,-7.61%,-3.92%,-9.08%,nan%


### P_Omega performance

An example of individual portfolio performance inquiry. 

In [11]:
pp['P_Omega'].port_perf(fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
P_Omega,8.48,-22.85,0.371106,2018-12-24,2018-10-03,2019-08-28,329
VGT,20.22,-35.07,0.576595,2022-10-14,2021-12-27,2023-11-20,693
XLV,14.36,-28.40,0.505501,2020-03-23,2020-01-22,2020-07-15,175
VHT,14.28,-28.85,0.494752,2020-03-23,2020-02-19,2020-06-08,110
GLD,2.29,-42.11,0.054485,2015-12-17,2012-10-04,2020-07-22,2848
TLT,0.56,-48.35,0.011678,2023-10-19,2020-08-04,NaN,1332


### P_Omega drawdowns

In [12]:
pp['P_Omega'].port_drawdown(fancy=True)

,DD,Date,Start,End,NrDays
No,,,,,
1,-22.85,2018-12-24,2018-10-03,2019-08-28,329
2,-19.75,2015-09-28,2015-07-20,2016-06-28,344
3,-19.66,2023-10-03,2021-12-29,NaN,820
4,-14.01,2020-03-18,2020-03-06,2020-04-06,31
5,-11.07,2016-12-01,2016-07-29,2017-04-24,269


### P_Omega weights

In [13]:
pp['P_Omega'].get_weights(fancy=True)

,Droll,Dfix,GLD,TLT,XLV,VGT,VHT
0,2015-06-25,2015-06-24,1.72,0.00,0.00,0.00,98.28
1,2015-09-25,2015-09-24,0.00,3.46,0.00,0.00,96.54
2,2015-12-28,2015-12-24,0.00,20.67,0.00,66.87,12.46
3,2016-03-28,2016-03-24,0.00,100.00,0.00,0.00,0.00
4,2016-06-27,2016-06-24,0.00,76.42,0.00,23.58,0.00
5,2016-09-27,2016-09-26,0.00,70.12,0.00,29.88,0.00
6,2016-12-27,2016-12-23,36.45,0.00,0.00,63.55,0.00
7,2017-03-28,2017-03-27,0.00,0.00,0.00,100.00,0.00
8,2017-06-27,2017-06-26,0.00,0.00,0.00,100.00,0.00
9,2017-09-26,2017-09-25,0.00,0.00,0.00,100.00,0.00
